In [2]:


import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from stable_baselines3 import SAC
from stable_baselines3.sac.policies import SACPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.logger import configure


class EpisodicRewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.cumulative_reward = 0.0

    def step(self, action):
        result = self.env.step(action)
        if len(result) == 5:
            obs, reward, terminated, truncated, info = result
            done = terminated or truncated
        else:
            obs, reward, done, info = result
        self.cumulative_reward += reward
        if done:
            reward = self.cumulative_reward
            self.cumulative_reward = 0.0
        else:
            reward = 0.0
        return obs, reward, done, info

    def reset(self, **kwargs):
        self.cumulative_reward = 0.0
        result = self.env.reset(**kwargs)
        if isinstance(result, tuple):
            obs, _ = result
        else:
            obs = result
        return obs


class TrajectoryReplay:
    def __init__(self):
        self.trajectories = []

    def add_trajectory(self, traj):
        self.trajectories.append(traj)

    def sample(self, batch_size):

        indices = np.random.randint(0, len(self.trajectories), size=batch_size)
        return [self.trajectories[i] for i in indices]

    def __len__(self):
        return len(self.trajectories)


class RewardModel(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_size=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim + action_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )

    def forward(self, s, a):
        x = torch.cat([s, a], dim=-1)
        return self.net(x)


def sample_subsequence_indices(traj_length, K):
    if K > traj_length:
        K = traj_length
    return np.random.choice(traj_length, size=K, replace=False)

def train_reward_model_rrd(reward_model, optimizer, trajectories, K=64, device='cpu'):
    reward_model.train()
    M = len(trajectories)

    loss_sum = 0.0
    for traj in trajectories:
        R_ep = sum([t[2] for t in traj])
        s_list = []
        a_list = []
        for (s, a, r, s_next, done) in traj:
            s_list.append(s)
            a_list.append(a)

        states = torch.tensor(s_list, dtype=torch.float, device=device)
        actions = torch.tensor(a_list, dtype=torch.float, device=device)

        Tj = len(traj)
        idx_subseq = sample_subsequence_indices(Tj, K)

        R_sa_sub = reward_model(states[idx_subseq], actions[idx_subseq])
        sum_R_sa_sub = R_sa_sub.sum()

        diff = (R_ep - sum_R_sa_sub) ** 2
        loss_sum += diff

    loss = loss_sum / M
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

def collect_episodes(env, model, n_episodes, device='cpu'):
    trajectories = []
    for _ in range(n_episodes):
        result = env.reset()
        if isinstance(result, tuple):
            obs, _ = result
        else:
            obs = result
        done = False
        traj = []
        while not done:
            if obs is None or (hasattr(obs, '__len__') and len(obs) == 0):
                raise ValueError("Observation is empty, check env.reset() output")
            obs_tensor = torch.tensor(obs, dtype=torch.float, device=device).unsqueeze(0)
            with torch.no_grad():
                action, _states = model.predict(obs_tensor.cpu().numpy(), deterministic=False)
            result = env.step(action[0])
            if len(result) == 5:
                next_obs, reward, terminated, truncated, info = result
                done = terminated or truncated
            else:
                next_obs, reward, done, info = result
            traj.append((obs, action[0], reward, next_obs, done))
            obs = next_obs
        trajectories.append(traj)
    return trajectories

def add_shaped_transitions_to_replay(model, reward_model, trajectories, gamma=0.99, device='cpu'):
    for traj in trajectories:
        states = []
        actions = []
        next_states = []
        dones = []
        for (s, a, r, s_next, d) in traj:
            states.append(s)
            actions.append(a)
            next_states.append(s_next)
            dones.append(d)

        s_tensor = torch.tensor(states, dtype=torch.float, device=device)
        a_tensor = torch.tensor(actions, dtype=torch.float, device=device)

        with torch.no_grad():
            shaped_rewards = reward_model(s_tensor, a_tensor).cpu().numpy().squeeze(-1)

        for i in range(len(traj)):
            s, a, _, s_next, d = traj[i]
            r_shaped = shaped_rewards[i]
            model.replay_buffer.add(
                s, s_next, a, r_shaped, d, infos=[{}],
            )



def main():
    class Args:
        env = "BipedalWalker-v3"
        episodes = 8000
        steps_per_update = 200
        rrd_k = 64
        device = "cuda"
    
    args = Args()

    device = torch.device(args.device)

    base_env = gym.make(args.env)
    env = EpisodicRewardWrapper(base_env)
    vec_env = DummyVecEnv([lambda: env])

    policy_kwargs = dict(
        net_arch=dict(pi=[64, 64], qf=[64, 64])
    )
    model = SAC(
        policy="MlpPolicy",
        env=vec_env,
        policy_kwargs=policy_kwargs,
        verbose=1,
        seed=42,
        buffer_size=50000,
        learning_starts=0,
        train_freq=1,
        gradient_steps=0,
        batch_size=512,
        gamma=0.99,
        device=device
    )
    model._logger = configure(folder=None, format_strings=["stdout"])

    state_dim = base_env.observation_space.shape[0]
    action_dim = base_env.action_space.shape[0]
    reward_model = RewardModel(state_dim, action_dim).to(device)
    reward_optimizer = optim.Adam(reward_model.parameters(), lr=3e-4)

    traj_replay = TrajectoryReplay()

    ep_record = []
    for ep in tqdm(range(args.episodes)):
        new_trajectories = collect_episodes(env, model, n_episodes=1, device=device)
        for traj in new_trajectories:
            traj_replay.add_trajectory(traj)

        if len(traj_replay) >= 5:
            batch_size_trajectories = 4
            sampled_trajs = traj_replay.sample(batch_size_trajectories)
            rrd_loss = train_reward_model_rrd(
                reward_model, reward_optimizer, sampled_trajs,
                K=args.rrd_k, device=device
            )

        add_shaped_transitions_to_replay(model, reward_model, new_trajectories, device=device)
        if len(traj_replay) >= 5:
            model.train(args.steps_per_update)

        ep_return = sum([t[2] for t in new_trajectories[-1]])
        ep_record.append(ep_return)
        
        if ep % 10 == 0:
            print(f"Episode {ep+1}/{args.episodes}, real env return = {ep_return:.2f}")

    print("Training finished.")
    return ep_record

In [3]:
train1 = main()

Using cuda device


  0%|                                          | 2/8000 [00:00<19:22,  6.88it/s]

Episode 1/8000, real env return = -104.00


  0%|                                       | 11/8000 [00:09<2:30:32,  1.13s/it]

Episode 11/8000, real env return = -98.50


...

Episode 7981/8000, real env return = -100.15


100%|████████████████████████████████████▉| 7991/8000 [2:35:46<00:10,  1.18s/it]

Episode 7991/8000, real env return = -100.17


100%|█████████████████████████████████████| 8000/8000 [2:35:56<00:00,  1.17s/it]

Training finished.


In [4]:
train2 = main()

Using cuda device


  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Episode 1/8000, real env return = -104.69


  0%|                                       | 11/8000 [00:08<1:58:06,  1.13it/s]

Episode 11/8000, real env return = -111.12


...

Episode 7981/8000, real env return = -102.69


100%|████████████████████████████████████▉| 7991/8000 [1:58:16<00:07,  1.15it/s]

Episode 7991/8000, real env return = -102.63


100%|█████████████████████████████████████| 8000/8000 [1:58:24<00:00,  1.13it/s]

Training finished.


In [5]:
train3 = main()

Using cuda device


  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Episode 1/8000, real env return = -103.57


  0%|                                       | 11/8000 [00:12<3:43:07,  1.68s/it]

Episode 11/8000, real env return = -94.72


...

Episode 7981/8000, real env return = -106.30


100%|████████████████████████████████████▉| 7991/8000 [1:58:09<00:07,  1.13it/s]

Episode 7991/8000, real env return = -106.69


100%|█████████████████████████████████████| 8000/8000 [1:58:16<00:00,  1.13it/s]

Training finished.


In [6]:
train4 = main()

Using cuda device


  0%|                                          | 2/8000 [00:00<07:48, 17.08it/s]

Episode 1/8000, real env return = -103.64


  0%|                                       | 11/8000 [00:10<2:41:47,  1.22s/it]

Episode 11/8000, real env return = -79.79


...

Episode 7981/8000, real env return = -106.10


100%|████████████████████████████████████▉| 7991/8000 [1:57:35<00:07,  1.16it/s]

Episode 7991/8000, real env return = -106.62


100%|█████████████████████████████████████| 8000/8000 [1:57:43<00:00,  1.13it/s]

Training finished.


In [7]:
train5 = main()

Using cuda device


  0%|                                          | 2/8000 [00:00<08:55, 14.92it/s]

Episode 1/8000, real env return = -103.73


  0%|                                       | 11/8000 [00:10<2:34:07,  1.16s/it]

Episode 11/8000, real env return = -109.32


...

Episode 7981/8000, real env return = -99.70


100%|████████████████████████████████████▉| 7991/8000 [1:59:51<00:07,  1.18it/s]

Episode 7991/8000, real env return = -100.00


100%|█████████████████████████████████████| 8000/8000 [1:59:59<00:00,  1.11it/s]

Training finished.
